In [ ]:
def create_deviation(deviations):
    return [np.random.randint(0, deviation) for deviation in deviations]

In [ ]:
from src.supply_chain_config import SupplyChainConfig
from src.dynamics import Dynamics
from src.evaluate import Evaluate
from src.policy import (
     MeanDemandPolicy,
     StaticBaseStockPolicyRandom,
     StaticBaseStockPolicyShortfall,
     EchelonBaseStockPolicy,
     sSPolicy,
     DualIndexPolicy,
     DynamicPolicy
)
import numpy as np
import pandas as pd
import random # Make sure random is imported
import math 

def create_deviation(deviations):
    return [np.random.randint(0, deviation) for deviation in deviations]

# 1. Simulation params (adjusted structure)
num_config_scenarios = 100 # Aantal verschillende willekeurige configuraties
num_demand_patterns_per_config = 100 # Aantal demand patterns per configuratie 
num_trajectories = 1 # Zoals in jouw code (1 traject per demand pattern run)
periods_per_trajectory = 200 # 

data_folder = "data"
output_filename = f"{data_folder}/simulated_data_random_configs.xlsx" # Output file naam

# Policies 
hedge_random = 0.4
hedge_shortfall = 0.4
hedge_echelon = 0.6

# Define randomization parameters (Standard deviation relative to base value)
# Adjust these percentages based on how much variation you want
lt_std_dev_perc = 0.10 # 10% standard deviation for Lead Times
cap_std_dev_perc = 0.10 # 10% standard deviation for Capacities
hc_std_dev_perc = 0.15 # 15% standard deviation for Holding Costs
bp_std_dev_perc = 0.20 # 20% standard deviation for Backlog Penalties
prod_demand_mean_std_dev_perc = 0.15 # 15% standard deviation for Product Demand Means

ar_params_std_dev_perc = 0.10 # 10% rel. std dev voor AR coëfficiënten 
sigma_std_dev_perc = 0.20 # 20% rel. std dev voor AR error sigma
kappa_std_dev_perc = 0.15 # 15% rel. std dev voor Gamma Kappa parameters
theta_std_dev_perc = 0.15 # 15% rel. std dev voor Gamma Theta parameters

all_results = []

# --- HOOFD LUS VOOR WILLEKEURIGE CONFIGURATIES ---
for config_run_id in range(num_config_scenarios):
    print(f"Running configuration scenario {config_run_id + 1}/{num_config_scenarios}")

    # 1. Initialize environment - READ BASE CONFIG
    # Read the original config to get base values
    base_sc = SupplyChainConfig() # Reads from Excel

    # 2. Randomize Configuration Parameters
    current_sc_config = {
        'Config_ID': config_run_id
    }

    # Randomize Lead Times (often integers, must be >= 1)
    randomized_lead_times = []
    for lt in base_sc.lead_times:
        # Add normal noise, ensure >= 1, round to nearest integer
        noise = np.random.normal(0, lt * lt_std_dev_perc)
        new_lt = round(max(1, lt + noise))
        randomized_lead_times.append(new_lt)
    base_sc.lead_times = randomized_lead_times # Update the sc object
    # Add to config dict for saving
    for i in range(base_sc.num_components):
        current_sc_config[f'LT{i+1}'] = randomized_lead_times[i]


    # Randomize Capacities (often integers, must be >= 0 or >= minimum batch size if applicable)
    randomized_capacities = []
    for cap in base_sc.capacities:
         # Add normal noise, ensure >= 0, round to nearest integer (or handle minimums)
         noise = np.random.normal(0, cap * cap_std_dev_perc)
         new_cap = round(max(0, cap + noise)) # Assuming capacity can be 0
         randomized_capacities.append(new_cap)
    base_sc.capacities = randomized_capacities # Update the sc object
    # Add to config dict for saving
    for i in range(base_sc.num_nodes):
        current_sc_config[f'CapN{i+1}'] = randomized_capacities[i]


    # Randomize Holding Costs (floats, must be >= 0)
    randomized_h_costs = []
    for h in base_sc.h:
        noise = np.random.normal(0, h * hc_std_dev_perc)
        new_h = max(0.0, h + noise)
        randomized_h_costs.append(new_h)
    base_sc.h = randomized_h_costs # Update the sc object
    # Add to config dict for saving
    for i in range(base_sc.num_components):
        current_sc_config[f'HC_C{i+1}'] = randomized_h_costs[i]


    # Randomize Backlog Penalties (floats, must be >= 0)
    randomized_p_penalties = []
    for p in base_sc.p:
        noise = np.random.normal(0, p * bp_std_dev_perc)
        new_p = max(0.0, p + noise)
        randomized_p_penalties.append(new_p)
    base_sc.p = randomized_p_penalties # Update the sc object
    # Add to config dict for saving (mapping to C numbers)
    for i in range(base_sc.num_products):
         current_sc_config[f'BP_C{base_sc.num_sub_components + i + 1}'] = randomized_p_penalties[i]


    # --- END Randomize Supply Chain Parameters ---


    # --- Randomize Demand Model Parameters ---

    # Randomize Aggregate Demand AR Parameters
    randomized_ar_params = []
    for ar_param in base_sc.aggregate_demand_ar_params:
        # AR params kunnen zowel positief als negatief zijn
        noise = np.random.normal(0, abs(ar_param) * ar_params_std_dev_perc) if ar_param != 0 else np.random.normal(0, 0.01)
        new_ar_param = ar_param + noise
        randomized_ar_params.append(new_ar_param)
    base_sc.aggregate_demand_ar_params = randomized_ar_params
    # Add to config dict for saving
    for i in range(base_sc.aggregate_demand_num_lags):
        current_sc_config[f'AR_Param_{i+1}'] = randomized_ar_params[i]


    # Randomize Aggregate Demand Error Sigma (must be > 0)
    sigma = base_sc.aggregate_demand_error_sigma
    noise = np.random.normal(0, sigma * sigma_std_dev_perc) if sigma > 0 else np.random.normal(0, 0.1)
    randomized_sigma = max(0.01, sigma + noise) # Ensure sigma is positive and not too close to zero
    base_sc.aggregate_demand_error_sigma = randomized_sigma
    current_sc_config['AR_Error_Sigma'] = randomized_sigma


    # Randomize Product Demand Means (floats, must be >= 0) - Deze had je al
    randomized_product_demand_mean = []
    for mean in base_sc.product_demand_mean:
        noise = np.random.normal(0, mean * prod_demand_mean_std_dev_perc) if mean > 0 else np.random.normal(0, 0.1)
        new_mean = max(0.0, mean + noise)
        randomized_product_demand_mean.append(new_mean)
    base_sc.product_demand_mean = randomized_product_demand_mean

    # Re-calculate Aggregate Demand Mean based on new product means
    base_sc.aggregate_demand_mean = sum(randomized_product_demand_mean)
    base_sc.avg_demand_per_product = randomized_product_demand_mean # Update this too

    # Add demand means to config dict for saving (Deze had je al)
    current_sc_config['AggDemandMean'] = base_sc.aggregate_demand_mean
    for i in range(base_sc.num_products):
         current_sc_config[f'MeanDemand_C{base_sc.num_components - base_sc.num_products + i + 1}'] = randomized_product_demand_mean[i]


    # Randomize Product Demand Split Kappa (floats, must be > 0)
    randomized_kappa = []
    for kappa in base_sc.product_demand_split_kappa:
        noise = np.random.normal(0, kappa * kappa_std_dev_perc) if kappa > 0 else np.random.normal(0, 0.1)
        new_kappa = max(0.01, kappa + noise) # Ensure kappa is positive and not too close to zero
        randomized_kappa.append(new_kappa)
    base_sc.product_demand_split_kappa = randomized_kappa
    # Add to config dict for saving
    for i in range(base_sc.num_products):
        current_sc_config[f'Kappa_P{i+1}'] = randomized_kappa[i]


    # Randomize Product Demand Split Theta (floats, must be > 0)
    randomized_theta = []
    for theta in base_sc.product_demand_split_theta:
        noise = np.random.normal(0, theta * theta_std_dev_perc) if theta > 0 else np.random.normal(0, 0.01)
        new_theta = max(0.01, theta + noise) # Ensure theta is positive and not too close to zero
        randomized_theta.append(new_theta)
    base_sc.product_demand_split_theta = randomized_theta
    # Add to config dict for saving
    for i in range(base_sc.num_products):
        current_sc_config[f'Theta_P{i+1}'] = randomized_theta[i]

    # --- END Randomize Demand Model Parameters ---

    # Now that sc is randomized, initialize Dynamics and Evaluate with THIS config
    dyn = Dynamics(base_sc)
    evalr = Evaluate(base_sc, dyn, num_trajectories, periods_per_trajectory)

    # --- MAAK DE BELEIDSLIJNEN NU AAN MET DE VOORBEREIDE CONFIGURATIE ---
    # Dit lost de AttributeError op
    policies = [
        StaticBaseStockPolicyRandom(base_sc, hedge_random),
        EchelonBaseStockPolicy(base_sc, hedge_echelon),
        StaticBaseStockPolicyShortfall(base_sc, hedge_shortfall)
        # Voeg andere policies hier toe met base_sc
    ]
    num_policies = len(policies) # Definieer hier het aantal policies

    # Map voor policy_id -> naam, moet overeenkomen met de volgorde in de 'policies' lijst hierboven
    policy_id_to_name = {
        0: 'policy_random',
        1: 'policy_echelon',
        2: 'policy_shortfall',
        # 3: 'policy_dynamic' # Voeg toe indien gebruikt
    }


    # --- INNER LUS VOOR DEMAND PATTERNS (voor DEZE configuratie) ---
    for dp in range(num_demand_patterns_per_config):
         # --- ADD TRY...EXCEPT BLOCK HERE ---
         try:
             # compare_policies retourneert de gemiddelde kosten per policy voor 1 traject
             # De vraag wordt nu gegenereerd met de gerandomiseerde vraagmodel parameters
             costs = evalr.compare_policies(policies)

             # Assembleer policy parameters (bs_arr) - Alleen bij SUCCES
             bs_arr = []
             for p in policies:
                 policy_levels_to_append = None
                 if hasattr(p, "base_stock_levels"):
                     policy_levels_to_append = p.base_stock_levels
                 elif hasattr(p, "S"):
                      if isinstance(p.S, (list, np.ndarray)) and len(p.S) == base_sc.num_components:
                          policy_levels_to_append = p.S
                      else:
                          pass # Handle if needed
                 # Handle other policy parameters if needed

                 if policy_levels_to_append is None:
                      bs_arr.append([np.nan] * base_sc.num_components) # Fill with NaN if no levels found
                 elif isinstance(policy_levels_to_append, (list, np.ndarray)) and len(policy_levels_to_append) == base_sc.num_components:
                      bs_arr.append(policy_levels_to_append)
                 else:
                      bs_arr.append([np.nan] * base_sc.num_components)

             bs_arr = np.array(bs_arr) # [num_policies x num_components]


             # --- COLLECT DATA FOR EACH POLICY (SUCCESS CASE) ---
             for i, policy_cost in enumerate(costs):
                 row_data = {}
                 row_data['Config_ID'] = config_run_id
                 row_data['demand_pattern'] = dp
                 row_data['policy_id'] = policy_id_to_name[i] # Use name directly

                 # Add policy parameters (from bs_arr)
                 for j in range(base_sc.num_components):
                     row_data[f'C{j+1}'] = bs_arr[i, j] # bs_arr[policy_index, component_index]

                 row_data['cost'] = policy_cost

                 # Add the randomized config parameters
                 row_data.update(current_sc_config)

                 all_results.append(row_data)

         except Exception as e:
             # --- HANDLE THE EXCEPTION ---
             if str(e) == "Action is not feasible":
                 print(f"   Simulation failed due to infeasible action for Config {config_run_id}, Demand Pattern {dp}. Logging as NaN.")
                 # --- COLLECT DATA FOR EACH POLICY (FAILURE CASE) ---
                 # Log NaN/0 for all policies for this failed run
                 for i, policy in enumerate(policies): # Loop through policies to create rows
                      row_data = {}
                      row_data['Config_ID'] = config_run_id
                      row_data['demand_pattern'] = dp
                      row_data['policy_id'] = policy_id_to_name[i] # Use name

                      # Set Policy Parameters (C columns) to NaN or 0 as requested
                      # Using NaN to indicate invalid/missing data due to failure
                      for j in range(base_sc.num_components):
                          # User asked for 0, but NaN is more appropriate for invalid data
                          # Let's use NaN as it doesn't skew averages later
                          row_data[f'C{j+1}'] = np.nan # Or 0 if user insists

                      # Set cost to NaN to indicate failure
                      # User asked for 0, but NaN is more appropriate for invalid cost
                      # Let's use NaN
                      row_data['cost'] = np.nan # Or 0 if user insists

                      # Add the randomized config parameters (These were the ones that caused failure)
                      row_data.update(current_sc_config)

                      all_results.append(row_data)

             else:
                 # If it's a different error, re-raise it
                 print(f"An unexpected error occurred in Config {config_run_id}, Demand Pattern {dp}: {e}")
                 raise e # Re-raise the exception


    # --- INNER LUS EINDE ---
# --- HOOFD LUS EINDE ---

# --- 7. BUILD FINAL DATAFRAME AND SAVE ---
print("Finished all simulations. Building final DataFrame.")
df = pd.DataFrame(all_results)
print(df)
df.to_excel("data/randomconfig100,100.xlsx", index=False, header=True)

Running configuration scenario 1/100
Running configuration scenario 2/100
Running configuration scenario 3/100
Running configuration scenario 4/100
Running configuration scenario 5/100
Running configuration scenario 6/100
Running configuration scenario 7/100
Running configuration scenario 8/100
Running configuration scenario 9/100
Running configuration scenario 10/100
Running configuration scenario 11/100
Running configuration scenario 12/100
Running configuration scenario 13/100
Running configuration scenario 14/100
Running configuration scenario 15/100
Running configuration scenario 16/100
Running configuration scenario 17/100
Running configuration scenario 18/100
Running configuration scenario 19/100
Running configuration scenario 20/100
Running configuration scenario 21/100
Running configuration scenario 22/100
Running configuration scenario 23/100
Running configuration scenario 24/100
Running configuration scenario 25/100
Running configuration scenario 26/100
Running configuration

In [7]:
df.to_excel("data/randomconfig100,1001.xlsx", index=False, header=True)